## Тестирование производительности OLAP-хранилищ Clickhouse и Vertica

В рамках исследования были рассмотрены Clickhouse и Vertica.

### Запуск

In [40]:
!docker-compose up -d

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Creating network "olap_default" with the default driver
Creating volume "olap_ugc_bench_clickhouse_data" with default driver
Creating volume "olap_ugc_bench_vertica_data" with default driver
Creating olap_clickhouse_1 ... 
Creating olap_vertica_1    ... 
ting olap_clickhouse_1 ... done

### Загрузка тестовых данных

Структура тестовой таблицы


| ----------- | ----------- |
| id      | UUID       |
| user_id   | UUID        |
| movie_id   | UUID        |
| viewed_frame   | Int        |

In [41]:
!export PYTHONPATH="${PYTHONPATH}:${PWD}/../.."

#### Загрузка тестовых данных в Clickhouse

In [42]:
from multiprocessing import Pool

import tqdm as tqdm
from clickhouse_driver import Client

from utils.fake_data_gen import Row, generate_fake_data
from config import CLICKHOUSE_HOST, NUMBER_OF_BATCHES, UPLOAD_BATCH_SIZE

client = Client(CLICKHOUSE_HOST)


def upload_batch(batch):
    columns = ', '.join(Row._fields)
    client.execute(
        f'INSERT INTO views ({columns}) VALUES',
        batch
    )

In [43]:
test_data = generate_fake_data(UPLOAD_BATCH_SIZE, NUMBER_OF_BATCHES)

with Pool() as pool:
    r = list(tqdm.tqdm(
        pool.imap(upload_batch, test_data),
        total=NUMBER_OF_BATCHES
    ))

100%|██████████| 1000/1000 [00:49<00:00, 20.08it/s]


#### Загрузка тестовых данных в Vertica

In [44]:
from multiprocessing import Pool

import tqdm
import vertica_python

from utils.fake_data_gen import Row, generate_fake_data
from config import NUMBER_OF_BATCHES, UPLOAD_BATCH_SIZE, VERTICA_CONNECTION_PARAMS


def upload_batch(batch):
    with vertica_python.connect(**VERTICA_CONNECTION_PARAMS) as connection:
        columns = ', '.join(Row._fields)
        placeholders = ', '.join(['%s'] * len(Row._fields))
        cursor = connection.cursor()
        cursor.executemany(
            f'INSERT INTO views ({columns}) VALUES ({placeholders})',
            batch
        )

In [45]:
test_data = generate_fake_data(UPLOAD_BATCH_SIZE, NUMBER_OF_BATCHES)

with Pool() as pool:
    r = list(tqdm.tqdm(
        pool.imap(upload_batch, test_data),
        total=NUMBER_OF_BATCHES
    ))

100%|██████████| 1000/1000 [01:35<00:00, 10.51it/s]


### Выполнение тестовых запросов

Использовались следующие тестовые запросы:

In [46]:
from utils.test_queries import QUERIES

for name, query in QUERIES.items():
    print(f'{name}')
    print(f'{query}')
    print("----------")

average_movies_per_user
select avg(movies_watched) from ( select count(movie_id) as movies_watched from views group by user_id    ) as movies_count;
----------
average_view_times
select avg(viewed_frame) from views;
----------
top_20_users_by_total_view_time
select user_id, sum(viewed_frame) as view_time from views group by user_id order by view_time desc limit 20;
----------
top_20_movies_by_view_time
select movie_id, max(viewed_frame) as view_time from views group by movie_id order by view_time desc limit 20;
----------
unique_movies_count
select count(distinct movie_id) from views;
----------
unique_users_count
select count(distinct user_id) from views;
----------


#### Тестирование Clickhouse

In [47]:
from clickhouse_driver import Client

from utils.test_queries import QUERIES
from utils.timer import timer
from config import BENCHMARK_ITERATIONS, CLICKHOUSE_HOST

client = Client(CLICKHOUSE_HOST)


@timer(BENCHMARK_ITERATIONS)
def execute_query(query: str):
    client.execute(query)

In [48]:
for name, query in QUERIES.items():
    print(f'{name}')
    execute_query(query)

average_movies_per_user
Average execution time (over 10 runs): 0.3282 seconds

average_view_times
Average execution time (over 10 runs): 0.0179 seconds

top_20_users_by_total_view_time
Average execution time (over 10 runs): 0.2414 seconds

top_20_movies_by_view_time
Average execution time (over 10 runs): 0.1241 seconds

unique_movies_count
Average execution time (over 10 runs): 0.0636 seconds

unique_users_count
Average execution time (over 10 runs): 0.1454 seconds



#### Тестирование Vertica

In [49]:
import vertica_python

from utils.test_queries import QUERIES
from utils.timer import timer
from config import BENCHMARK_ITERATIONS, VERTICA_CONNECTION_PARAMS


@timer(BENCHMARK_ITERATIONS)
def execute_query(query: str):
    with vertica_python.connect(**VERTICA_CONNECTION_PARAMS) as connection:
        cursor = connection.cursor()
        cursor.execute(query)

In [50]:
for name, query in QUERIES.items():
    print(f'{name}')
    execute_query(query)

average_movies_per_user
Average execution time (over 10 runs): 0.4929 seconds

average_view_times
Average execution time (over 10 runs): 0.2027 seconds

top_20_users_by_total_view_time
Average execution time (over 10 runs): 0.4802 seconds

top_20_movies_by_view_time
Average execution time (over 10 runs): 1.2334 seconds

unique_movies_count
Average execution time (over 10 runs): 0.6284 seconds

unique_users_count
Average execution time (over 10 runs): 0.4870 seconds



### Вывод

Clickhouse показывает **лучшую производительность** во всех проведенных тестах!

### Остановка

In [53]:
!docker-compose down -v

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Stopping olap_vertica_1    ... 
Stopping olap_clickhouse_1 ... 
ping olap_vertica_1    ... doneRemoving olap_vertica_1    ... 
Removing olap_clickhouse_1 ... 
Removing network olap_default32mdone
Removing volume olap_ugc_bench_clickhouse_data
Removing volume olap_ugc_bench_vertica_data
